In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,accuracy_score
from sklearn.preprocessing import StandardScaler

In [2]:
iris = load_iris()
X = iris.data 
y = iris.target 

In [3]:
scaler=StandardScaler()
X=scaler.fit_transform(X)

In [4]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

In [5]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [6]:
def linear_regression(X, weights, bias):
    return np.dot(X, weights) + bias

In [7]:
def log_likelihood(y_true, y_pred):
    # Clip y_pred to avoid log(0) errors
    y_pred = np.clip(y_pred, 1e-7, 1 - 1e-7)
    return np.sum(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))


In [8]:
def gradient_descent(X, y_true, y_pred, weights, bias, learning_rate):
    n = X.shape[0]
    dw = np.dot(X.T, (y_pred - y_true)) / n
    db = np.sum(y_pred - y_true) / n

    weights -= learning_rate * dw
    bias -= learning_rate * db

    return weights, bias


In [15]:
def one_vs_rest_logistic_regression(X, y, num_classes, learning_rate=0.1, iterations=1000):
    weights = np.zeros((num_classes, X.shape[1]))
    biases = np.zeros(num_classes)
    
    for i in range(num_classes):
        y_binary = np.where(y == i, 1, 0)
        
        w = np.zeros(X.shape[1])
        b = 0
        
        for _ in range(iterations):
            linear_output = linear_regression(X, w, b)
            
            y_pred = sigmoid(linear_output)
            
            loss = log_likelihood(y_binary, y_pred)
            
            w, b = gradient_descent(X, y_binary, y_pred, w, b, learning_rate)
        
        weights[i, :] = w
        biases[i] = b
    
    return weights, biases


In [10]:
def predict(X, weights, biases):
    linear_outputs = np.dot(X, weights.T) + biases
    probabilities = sigmoid(linear_outputs)
    return np.argmax(probabilities, axis=1)


In [16]:
num_classes = len(np.unique(y_train))

weights, biases = one_vs_rest_logistic_regression(X_train, y_train, num_classes, learning_rate=0.1, iterations=1000)

y_pred = predict(X_test, weights, biases)

accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")


Model Accuracy: 91.11%


In [12]:
r2 = r2_score(y_test, y_pred)
print("r2",r2)

r2 0.8717948717948718
